In [ ]:
# Boto3 SageMaker Invoke Endpoint
# This example shows how to invoke SageMaker Endpoint from outside of AWS environment using Boto3 SDK
# Boto is the Amazon Web Services (AWS) SDK for Python
# https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

# Endpoint: XGBoost - Kaggle Bike Rental - Regressor Trained in XGBoost Lectures
# Makesure Endpoint is deployed before running this example
# 
# Reference:
#  https://github.com/awslabs/amazon-sagemaker-examples

# NOTE: SageMaker SDK now requires additional permissions DescribeEndpoint, DescribeEndpointConfig in-addition to InvokeEndpoint
#   boto3 SDK requires just InvokeEndpoint permission.
#   Please update SageMakerInvokeEndpoint permissions to reflect this policy document:
#   Logon with my_admin account and update permissions (IAM->Policies->SageMakerInvokeEndpoint->Edit Policy)
#   
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "sagemaker:DescribeEndpointConfig",
                "sagemaker:DescribeEndpoint",
                "sagemaker:InvokeEndpoint"
            ],
            "Resource": "*"
        }
    ]
}

In [2]:
import boto3
import math
import dateutil
import re

In [3]:
# Import s3 bucket name from .env file

import os
env_vars = !cat ./.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key]=value

In [4]:
endpoint_name = os.environ['ENDPOINT_NAME']
profile_name = os.environ['PROFILE_NAME']
#print(endpoint_name)

In [5]:
# Establish a boto3 session with AWS

boto_session = boto3.Session(profile_name=profile_name, region_name='us-east-1')

In [6]:
print(boto_session.get_available_services())

['accessanalyzer', 'account', 'acm', 'acm-pca', 'alexaforbusiness', 'amp', 'amplify', 'amplifybackend', 'amplifyuibuilder', 'apigateway', 'apigatewaymanagementapi', 'apigatewayv2', 'appconfig', 'appconfigdata', 'appfabric', 'appflow', 'appintegrations', 'application-autoscaling', 'application-insights', 'applicationcostprofiler', 'appmesh', 'apprunner', 'appstream', 'appsync', 'arc-zonal-shift', 'artifact', 'athena', 'auditmanager', 'autoscaling', 'autoscaling-plans', 'b2bi', 'backup', 'backup-gateway', 'backupstorage', 'batch', 'bcm-data-exports', 'bedrock', 'bedrock-agent', 'bedrock-agent-runtime', 'bedrock-runtime', 'billingconductor', 'braket', 'budgets', 'ce', 'chatbot', 'chime', 'chime-sdk-identity', 'chime-sdk-media-pipelines', 'chime-sdk-meetings', 'chime-sdk-messaging', 'chime-sdk-voice', 'cleanrooms', 'cleanroomsml', 'cloud9', 'cloudcontrol', 'clouddirectory', 'cloudformation', 'cloudfront', 'cloudfront-keyvaluestore', 'cloudhsm', 'cloudhsmv2', 'cloudsearch', 'cloudsearchdoma

In [23]:
# Acquire Sagemaker runtime client
client = boto_session.client(service_name='sagemaker-runtime', region_name='us-east-1')

In [26]:
# Specify your endpoint name
endpoint_name = 'xgboost-bikerental-v1'

In [9]:
# Raw Data
#datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
# Actual=562
sample_one = '2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027'
# Actual=569
sample_two = '2012-12-19 18:00:00,4,0,1,1,15.58,19.695,50,23.9994'
# Actual=4
sample_three = '2012-12-10 01:00:00,4,0,1,2,14.76,18.94,100,0'

In [16]:
# Transform the Raw data to 

# season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour

def transform_data(data):
    features = data.split(',')

    #Extract year,month,day,dayofweek,hour
    dt = dateutil.parser.parse(features[0])

    features.append(str(dt.year))
    features.append(str(dt.month))
    features.append(str(dt.day))
    features.append(str(dt.weekday()))
    features.append(str(dt.hour))

    return ','.join(features[1:])

In [17]:
print('Raw data:',sample_one)
print('Transformed data:', transform_data(sample_one))

Raw data: 2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027
Transformed data: 4,0,1,1,16.4,20.455,50,26.0027,2012,12,19,2,17


In [27]:
client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body=transform_data(sample_one).encode('utf-8'), 
                       ContentType='text/csv')

{'ResponseMetadata': {'RequestId': '03d8eb7b-eb6f-46e3-a472-2e5d99a432df',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '03d8eb7b-eb6f-46e3-a472-2e5d99a432df',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Tue, 07 May 2024 00:32:01 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '18',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f869be0e250>}

In [48]:
result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body=transform_data(sample_one).encode('utf-8'), 
                       ContentType='text/csv')

In [49]:
result['Body']

In [50]:
result = result['Body'].read().decode('utf-8')

In [52]:
print(result)

6.349300861358643



In [55]:
# actual count for sample_one is 562

print('Predicted Count',math.expm1(float(result)))

Predicted Count 571.092597122044


In [57]:
print('\n'.join([transform_data(sample_one), transform_data(sample_two)]))

4,0,1,1,16.4,20.455,50,26.0027,2012,12,19,2,17
4,0,1,1,15.58,19.695,50,23.9994,2012,12,19,2,18


In [58]:
# prediction for multiple observations

result = client.invoke_endpoint(EndpointName=endpoint_name, 
                       Body=('\n'.join([transform_data(sample_one), transform_data(sample_two)]).encode('utf-8')), 
                       ContentType='text/csv')

In [59]:
result = result['Body'].read().decode('utf-8')

In [60]:
result

'6.349300861358643\n6.321451187133789\n'

In [67]:
#Batch Prediction

def run_predictions(data,batch_size):

    predictions = []
    transformed_data = [transform_data(row.strip()) for row in data]

    pattern = r'[^0-9.]+'

    for i in range(0, len(data),batch_size):
        print(i,i+batch_size)

        result = client.invoke_endpoint(EndpointName=endpoint_name,
                                            Body = ('\n'.join(transformed_data[i:i+batch_size]).encode('utf-8')),
                                            ContentType='text/csv')

        result = result['Body'].read().decode('utf-8')
        result = re.split(pattern,result)

        predictions += [math.expm1(float(r)) for r in result if r!=""]

    return predictions

In [68]:
run_predictions([sample_one, sample_two, sample_three], 10)

0 10


[571.092597122044, 555.3798181158465, 10.489585991183136]

In [70]:
# Run a batch prediction test.csv file

data = []
with open('test.csv','r') as f:
    
    f.readline()   # skip header
    data = f.readlines()

In [71]:
len(data)

6493

In [73]:
%%time
predictions = run_predictions(data, 100)

0 100
100 200
200 300
300 400
400 500
500 600
600 700
700 800
800 900
900 1000
1000 1100
1100 1200
1200 1300
1300 1400
1400 1500
1500 1600
1600 1700
1700 1800
1800 1900
1900 2000
2000 2100
2100 2200
2200 2300
2300 2400
2400 2500
2500 2600
2600 2700
2700 2800
2800 2900
2900 3000
3000 3100
3100 3200
3200 3300
3300 3400
3400 3500
3500 3600
3600 3700
3700 3800
3800 3900
3900 4000
4000 4100
4100 4200
4200 4300
4300 4400
4400 4500
4500 4600
4600 4700
4700 4800
4800 4900
4900 5000
5000 5100
5100 5200
5200 5300
5300 5400
5400 5500
5500 5600
5600 5700
5700 5800
5800 5900
5900 6000
6000 6100
6100 6200
6200 6300
6300 6400
6400 6500
CPU times: user 427 ms, sys: 10.6 ms, total: 437 ms
Wall time: 7.97 s


In [74]:
len(predictions), len(data)

(6493, 6493)